In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

pad_sequences = keras.preprocessing.sequence.pad_sequences

In [2]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
train_data, val_data, train_labels, val_labels = train_test_split(train_data, 
                                                                  train_labels, 
                                                                  test_size=0.30, 
                                                                  shuffle=True,
                                                                  random_state=0)

In [4]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [6]:
pad_length = 256

train_data = pad_sequences(train_data,
                           value=word_index["<PAD>"],
                           padding='post',
                           maxlen=pad_length)

val_data = pad_sequences(val_data,
                         value=word_index["<PAD>"],
                         padding='post',
                         maxlen=pad_length)

test_data = pad_sequences(test_data,
                          value=word_index["<PAD>"],
                          padding='post',
                          maxlen=pad_length)

In [7]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, RepeatVector, dot, multiply, Permute, Lambda

def attention(layer):
    # --- Attention is all you need --- #
    units = layer.shape[-1]  # Use the last dimension
    attention = Dense(1, activation='tanh')(layer)
    attention = Flatten()(attention)
    attention = tf.keras.activations.softmax(attention)
    attention = RepeatVector(units)(attention)
    attention = Permute([2, 1])(attention)
    representation = multiply([layer, attention])
    representation = Lambda(lambda x: tf.reduce_sum(x, axis=-2),
                            output_shape=(units,))(representation)
    # ---------------------------------- #
    return representation

In [8]:
def get_optimizer(option=0, learning_rate=0.001):
    if option == 0:
        return tf.keras.optimizers.Adam(learning_rate)
    elif option == 1:
        return tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)
    elif option == 2:
        return tf.keras.optimizers.RMSprop(learning_rate) 
    else:
        return tf.keras.optimizers.Adam(learning_rate)

In [9]:
layers = keras.layers
models = keras.models
    
def create_tunable_model(hp, vocab_size=10000, pad_length=256):

    # Instantiate model params
    embedding_size = hp.Int('embedding_size', min_value=8, max_value=512, step=8)
    spatial_dropout = hp.Float('spatial_dropout', min_value=0, max_value=0.5, step=0.05)

    conv_layers = hp.Int('conv_layers', min_value=1, max_value=5, step=1)
    rnn_layers = hp.Int('rnn_layers', min_value=1, max_value=5, step=1)
    dense_layers = hp.Int('dense_layers', min_value=1, max_value=3, step=1)

    conv_filters = hp.Int('conv_filters', min_value=32, max_value=512, step=32)
    conv_kernel = hp.Int('conv_kernel', min_value=1, max_value=8, step=1)

    concat_dropout = hp.Float('concat_dropout', min_value=0, max_value=0.5, step=0.05)
    dense_dropout = hp.Float('dense_dropout', min_value=0, max_value=0.5, step=0.05)

    inputs = layers.Input(name='inputs',shape=[pad_length])
    layer  = layers.Embedding(vocab_size, embedding_size)(inputs)
    layer  = layers.SpatialDropout1D(spatial_dropout)(layer)

    for l in range(conv_layers):
        if l==0:
            conv = layers.Conv1D(filters=conv_filters, kernel_size=conv_kernel, 
                                 padding='valid', kernel_initializer='he_uniform')(layer)
        else:
            conv = layers.Conv1D(filters=conv_filters, kernel_size=conv_kernel, 
                                 padding='valid', kernel_initializer='he_uniform')(conv) 

    avg_pool_conv = layers.GlobalAveragePooling1D()(conv)
    max_pool_conv = layers.GlobalMaxPooling1D()(conv)

    representations = list()
    for l in range(rnn_layers):
        
        use_bidirectional = hp.Choice(f'use_bidirectional_{l}', values=[0, 1])
        use_lstm = hp.Choice(f'use_lstm_{l}', values=[0, 1])
        units = hp.Int(f'units_{l}', min_value=8, max_value=512, step=8)

        if use_lstm == 1:
            rnl = layers.LSTM
        else:
            rnl = layers.GRU

        if use_bidirectional==1:
            layer = layers.Bidirectional(rnl(units, return_sequences=True))(layer)
        else:
            layer = rnl(units, return_sequences=True)(layer)

        representations.append(attention(layer))

    layer = layers.concatenate(representations + [avg_pool_conv, max_pool_conv])
    layer = layers.Dropout(concat_dropout)(layer)

    for l in range(dense_layers):
        dense_units = hp.Int(f'dense_units_{l}', min_value=8, max_value=512, step=8)
        layer = layers.Dense(dense_units)(layer)
        layer  = layers.LeakyReLU()(layer)
        layer = layers.Dropout(dense_dropout)(layer)

    layer  = layers.Dense(1, name='out_layer')(layer)
    outputs  = layers.Activation('sigmoid')(layer)

    model  = models.Model(inputs=inputs, outputs=outputs)

    hp_learning_rate = hp.Choice('learning_rate', values=[0.002, 0.001, 0.0005])
    optimizer_type = hp.Choice('optimizer', values=list(range(3)))
    optimizer = get_optimizer(option=optimizer_type, learning_rate=hp_learning_rate)
    
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    
    return model

In [10]:
import keras_tuner as kt

In [11]:
tuner = kt.BayesianOptimization(hypermodel=create_tunable_model,
                                objective='val_acc',
                                max_trials=100,
                                num_initial_points=3,
                                directory='storage',
                                project_name='imdb',
                                seed=42)

tuner.search(train_data, train_labels, 
             epochs=30,
             batch_size=64, 
             validation_data=(val_data, val_labels),
             shuffle=True,
             verbose=2,
             callbacks = [EarlyStopping('val_acc', patience=3, restore_best_weights=True)]
             )

Trial 100 Complete [00h 01m 24s]
val_acc: 0.9017333388328552

Best val_acc So Far: 0.9037333130836487
Total elapsed time: 06h 31m 41s


In [12]:
# use to visualize: https://github.com/lutzroeder/netron

In [13]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)

In [14]:
print(best_hps.values)

{'embedding_size': 512, 'spatial_dropout': 0.0, 'conv_layers': 1, 'rnn_layers': 5, 'dense_layers': 1, 'conv_filters': 512, 'conv_kernel': 8, 'concat_dropout': 0.0, 'dense_dropout': 0.0, 'use_bidirectional_0': 0, 'use_lstm_0': 0, 'units_0': 8, 'dense_units_0': 512, 'learning_rate': 0.0005, 'optimizer': 0, 'use_bidirectional_1': 1, 'use_lstm_1': 0, 'units_1': 8, 'dense_units_1': 8, 'dense_units_2': 512, 'use_bidirectional_2': 0, 'use_lstm_2': 0, 'units_2': 8, 'use_bidirectional_3': 0, 'use_lstm_3': 1, 'units_3': 512, 'use_bidirectional_4': 1, 'use_lstm_4': 0, 'units_4': 512}


In [15]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 256)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 256, 512)  │  5,120,000 │ inputs[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 256, 512)  │          0 │ embedding_1[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, 256, 8)    │     12,528 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 256, 16)   │        864 │ gru_1[0][0]       │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_3 (GRU)         │ (None, 256, 8)    │        624 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256, 512)  │  1,067,008 │ gru_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 256, 1024) │  3,151,872 │ lstm[0][0]        │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256, 1)    │          9 │ gru_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256, 1)    │         17 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256, 1)    │          9 │ gru_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256, 1)    │        513 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256, 1)    │      1,025 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 256)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 256)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 256)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_1 (Softmax) │ (None, 256)       │          0 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_2 (Softmax) │ (None, 256)       │          0 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_3 (Softmax) │ (None, 256)       │          0 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_4 (Softmax) │ (None, 256)       │          0 │ flatten_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_5 (Softmax) │ (None, 256)       │          0 │ flatten_5[0][0] 

 Total params: 12,780,262 (48.75 MB)

 Trainable params: 12,780,262 (48.75 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.save("best_model.h5")